
# Splice Transactions with Spark and JDBC

Now we'll get a glimpse into the transactional nature of Splice Machine.  We'll switch to the `%spark` interpreter, which uses the Scala programming language; this allows us to easily demonstrate a clean way to programmatically interact with Splice Machine via JDBC. In this demonstration, we'll:

<ul class="italic">
    <li>Run setup code to create our variables and define a simple Spark procedure named <code>checkBalance</code>.</li>
    <li>Use JDBC calls to set and check values from the database.</li>
    <li>Show the use of <code>commit</code> and <code>rollback</code> to control transactional persistence of our updates.</li>
</ul>


<p class="noteNote">This assumes that Notebook <code>3.2 The Life of a Query</code> is run, so that the table <code>SUPPLIER</code> exists.</p>

The first step is to run the next cell, which creates the spark context.


In [ ]:
%%scala
%%spark --start
SparkSession.builder

Next we connect to the database and then define a simple procedure, `checkBalance`, which checks and *pretty-prints* the current balance for a supplier:

In [ ]:
%%scala 
import java.sql.{DriverManager,Connection}
val jdbcurl : String = sys.env("BEAKERX_SQL_DEFAULT_JDBC")
var conn = DriverManager.getConnection(jdbcurl )
conn.setAutoCommit(false)
var statement = conn.createStatement()

def checkBalance(suppKey: Integer) : Unit = {
    var resSet = statement.executeQuery("SELECT S_ACCTBAL FROM SUPPLIER WHERE S_SUPPKEY = " + suppKey.toString())
    while ( resSet.next() ) {
        var bl = resSet.getString("S_ACCTBAL")
        println("================ Current account balance: " + bl + " ====================");
    }
}


Now we can check the balance easily at any time with the `checkBalance` call, passing in the supply key as a parameter. For example:

In [ ]:
%%scala 
checkBalance(6517)

## Committing a Transaction

Our setup code created two useful variables we can now employ:

<table class="splicezepNoBorder">
   <tbody>
      <tr>
          <td><code>statement</code></td>
          <td>The standard JDBC Statement object, on which we run functions such as <code>executeUpdate</code>.</td>
      </tr>
      <tr>
          <td><code>conn</code></td>
          <td>A standard JDBC Connection object, on which we can run <code>commit</code> and <code>rollback</code>.</td>
      </tr>
    </tbody>
</table>

The first example shows an update statement which, upon committing, retains the value to which it is set:

In [ ]:
%%scala 
statement.executeUpdate("update SUPPLIER SET S_ACCTBAL = 10000 WHERE S_SUPPKEY = 6517");
checkBalance(6517);
conn.commit();
checkBalance(6517);

## Rolling Back a Transaction

If we choose to do a rollback, the value returns to the previous value before the transactional context, as shown in the next example:


In [ ]:
%%scala 
statement.executeUpdate("update SUPPLIER SET S_ACCTBAL = 0 WHERE S_SUPPKEY = 6517")
checkBalance(6517);
conn.rollback();
checkBalance(6517);


## Where to Go Next

The next notebook in this presentation walks you through <a href="./3.5%20Creating%20Applications.ipynb">creating applications with Splice Machine.</a>
